
# Shark Tank

## imports

In [3]:
import pandas as pd
import seaborn as sns
import re
"""
import sys
sys.path.append('../')

import src.cl as fc"""

## load data

In [4]:
sh = pd.read_csv('./attacks.csv',encoding="ISO-8859-1")
#sh.head()

## Limpiar columnas

In [5]:
#fix col names
diccio_todas = {col: col.replace(" ","_")  for col in list(sh.columns)}
sh.rename(columns=diccio_todas, inplace=True)


In [6]:
#drop unnecessary cols
sh.drop(["Unnamed:_22","Unnamed:_23", "Case_Number.1","Case_Number.2","href_formula","href","pdf"], axis=1, inplace=True)

In [7]:
sh.shape

(25723, 17)

In [39]:
#work data frame
sh.head()

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,6299.0


## Clean the data
### NAN values


In [54]:
sh.isnull().sum()

Case_Number                  1
Date                         0
Year                         2
Type                         4
Country                     50
Area                       454
Location                   540
Activity                   542
Name                       209
Sex_                       564
Age                       2821
Injury                      28
Fatal_(Y/N)                535
Time                      3342
Species_                  2832
Investigator_or_Source      17
original_order               0
dtype: int64

## drop NaN rows

In [8]:
sh.dropna(axis=0, how = "all",inplace=True)

In [10]:
sh.shape

(8703, 17)

In [11]:
sh.drop_duplicates(inplace=True)

In [13]:
sh.shape

(6311, 17)

In [14]:
sh.drop_duplicates(subset=['Case_Number'],inplace=True)
sh.shape

(6288, 17)

## ask for odd cases *

In [15]:
sh.drop([6302,25722], axis=0, inplace=True)

In [18]:
sh.shape

(6286, 17)

## fill NaN values

In [17]:
sh.Year.interpolate(method='linear', limit_direction='forward', axis=0)

0       2018.0
1       2018.0
2       2018.0
3       2018.0
4       2018.0
         ...  
6297       0.0
6298       0.0
6299       0.0
6300       0.0
6301       0.0
Name: Year, Length: 6286, dtype: float64

### pick object columns  **

In [27]:
string = list(sh.select_dtypes(include="object").columns) 
print(string)

['Case_Number', 'Date', 'Type', 'Country', 'Area', 'Location', 'Activity', 'Name', 'Sex_', 'Age', 'Injury', 'Fatal_(Y/N)', 'Time', 'Species_', 'Investigator_or_Source']


In [26]:
sh[string].fillna("unknown", inplace=True)
sh.tail(6)

/home/alejandro/.local/lib/python3.8/site-packages/pandas/core/frame.py:5171: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
6298,ND.0004,Before 1903,0.0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,NaN,NaN,"H. Taunton; N. Bartlett, pp. 233-234",5.0
6299,ND.0003,1900-1905,0.0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,NaN,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",4.0
6300,ND.0002,1883-1889,0.0,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,NaN,NaN,"The Sun, 10/20/1938",3.0
6301,ND.0001,1845-1853,0.0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,15,"FATAL. ""Shark bit him in half, carrying away t...",Y,NaN,NaN,S.W. Baker,2.0
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## done the fools way

In [19]:
sh.fillna("unkown",inplace=True)

In [20]:
sh.Country.value_counts()

USA                       2222
AUSTRALIA                 1336
SOUTH AFRICA               577
PAPUA NEW GUINEA           134
NEW ZEALAND                127
                          ... 
MALDIVE ISLANDS              1
NICARAGUA                    1
NORTH SEA                    1
RED SEA / INDIAN OCEAN       1
CEYLON (SRI LANKA)           1
Name: Country, Length: 213, dtype: int64

## Invalid Type is unusable data

In [21]:
sh.Type.unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', 'unkown', 'Boat', 'Boatomg'], dtype=object)

In [22]:
sh = sh[sh["Type"] != "Invalid"]
sh.shape


(5743, 17)

In [26]:
sh.sample(6)

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex_,Age,Injury,Fatal_(Y/N),Time,Species_,Investigator_or_Source,original_order
3848,1962.01.14.a,14-Jan-1962,1962.0,Unprovoked,AUSTRALIA,New South Wales,"Lennox Head, Ballina",Surfing,Gary Raymond Hiscocks,M,19,Dorsum of foot lacerated & toe severed,N,13h30,"Bronze whaler shark, 2.1 m [7'] shark","The Star (Johannesburg), Daily Telegraph (Sydn...",2455.0
3956,1960.10.00.e,Oct-1960,1960.0,Unprovoked,PAPUA NEW GUINEA,New Britain,"East Nakanai, Talasea",Collecting shells,unkown,M,26,Injuries to leg & foot,N,unkown,unkown,"A.M. Rapson, p.149",2347.0
3652,1964.12.09,09-Dec-1964,1964.0,Unprovoked,PAPUA NEW GUINEA,Bougainville (North Solomons),"Off Sing Village, Buka Island","Canoe swamped, swimming back to canoe","Soso, a male",M,28,Survived,N,Late afternoon,2.1 m [7'] shark,"South Pacific Post (Port Moresby), 12/16/1964",2651.0
1717,2003.11.27,27-Nov-2003,2003.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Sodwana,Spearfishing,Seldon Jee,M,21,"Presumed FATAL, severed hand recovered",Y,>08h00,"Tiger shark, 4 m [13'] ?","E. Ritter, GSAF",4586.0
1035,2010.03.27,27-Mar-2010,2010.0,Unprovoked,REUNION,Saint-Benoit,Bittern,Surfing,Oliver Schorebreak,M,34,"No injury, board bitten",N,11h00,1.5 m shark,"C. Ekstander, GSAF",5268.0
146,2017.04.29.d,29-Apr-2017,2017.0,Unprovoked,USA,California,"San Onofre, San Diego County",Swimming,Leeanne Ericson,F,unkown,Major injury to posterior thigh,N,17h34,unkown,"R. Collier, GSAF",6157.0


## subset fecha formato

In [33]:
import src.date_f  

ModuleNotFoundError: No module named 'src.date_f'

In [31]:
sh["Date_format"] = sh.Date.apply(Date_f)
sh.loc[r"\d+\-\w{3}\-\d+"].shape

KeyError: '\\d+\\-\\w{3}\\-\\d+'

In [24]:
sh_d= sh[sh["Date"] == r"\w+" ]
sh_d.shape

(0, 17)

## nueva columna Moon_phase
### subset?

In [ ]:
sh["Moon_phase"] = sh.Date.apply(moon_phase)